# **🧬 생물정보학과 파이썬 💻**

* <img src="https://microbenotes.com/wp-content/uploads/2018/12/Bioinformatics-Introduction-and-Applications.jpg">

## Chapter 7: 서열 정렬 (Sequence Alignment)

### 🧬 서열 정렬을 알아보자~!

단백질·유전자 서열을 서로 맞춰보면 **보존 영역(conserved region)**과 변이 지점을 한눈에 파악할 수 있습니다.  
특히 **인슐린**처럼 종 간에 보존성이 높은 단백질의 경우, 정렬을 통해 **기능 핵심 잔기**를 추적하고,  
치료제 설계에서 **안정성·면역원성 예측**의 단서를 얻을 수 있습니다.  

이번 장에서는 **전역(Global)·국소(Local) 정렬**, **점수 체계(치환행렬·갭 페널티)**를 이해하고,  
인슐린 서열(발췌본)로 실습해 볼까요?

---

### 🎯 실습 목표

- 전역/국소 정렬의 차이와 점수 계산 방식(매치/미스매치/갭)을 설명할 수 있다.  
- Biopython을 이용해 **Pairwise 서열 정렬**을 수행하고 정렬 결과를 해석할 수 있다.  
- **치환 행렬(BLOSUM/PAM)**과 **갭 오픈/익스텐드** 조정을 통해 정렬 품질을 비교할 수 있다.  
- 정렬로부터 **일치율(% identity)**, **커버리지**, **보존 모티프**를 요약 리포트로 만들 수 있다.  

---

### 🧪 생명과학에서 서열 정렬이 중요한 이유

- **보존 모티프 식별**: 리간드 결합, 구조 안정화 등 **기능 핵심 부위** 파악  
- **종 간 비교**: 모델동물(쥐·돼지 등)과 사람 단백질의 **유사성/차이** 정량화  
- **치료제 설계**: 변이 도입 시 **기능 유지·면역원성 저감**을 위한 가이드  
- **진화 추적**: 서열 기반 **계통학적 관계**와 **선택압 탐색**의 출발점  

---


```python

## Chapter 7: 아미노산 서열 비교 분석 🧬  
### Needleman–Wunsch 알고리즘

인슐린은 사람과 동물에서 매우 **보존성이 높은 단백질**입니다.  
이 실습에서는 **NCBI** 데이터베이스에서  
사람·돼지·소·닭의 인슐린 단백질 서열을 직접 가져와 아미노산 서열을 분석합니다.
---

### 🔗 참고 링크

- [미국 국립생물공학정보센터 (NCBI)](https://www.ncbi.nlm.nih.gov/)  
- [Human Insulin (P01308)](https://www.ncbi.nlm.nih.gov/protein/P01308.1)  
- [Pig Insulin (P01315)](https://www.ncbi.nlm.nih.gov/protein/P01315.2)  
- [Cow Insulin (P01317)](https://www.ncbi.nlm.nih.gov/protein/P01317.2)  
- [Chicken Insulin (P67970)](https://www.ncbi.nlm.nih.gov/protein/P67970.1)

---

### 🎯 실습 목표

- 🔹 **NCBI**에서 단백질 서열을 불러올 수 있다.  
- 📄 **`SeqIO`와 `Entrez` 모듈**을 이용해 FASTA 형식 데이터를 읽을 수 있다.  
- 🔬 각 종의 인슐린 서열을 출력하여 **길이와 구성 차이**를 비교할 수 있다.  


```python

In [30]:
# ✅ 1단계: Biopython 설치
!pip install biopython


In [31]:
# ✅ 라이브러리 불러오기
from Bio import SeqIO, Entrez

# NCBI 데이터 접근을 위한 사용자 이메일 등록
Entrez.email = "cluid0416@naver.com"  # ← 반드시 본인 이메일로 변경


In [35]:
# ✅ NCBI에서 단백질 서열 불러오기

# 사람(Human)
human = Entrez.efetch(db="protein", id="P01308.1", rettype="fasta", retmode="text")
human_record = SeqIO.read(human, "fasta")

In [42]:
# 돼지(Pig)
pig = Entrez.efetch(db = "protein", id = "P01315.2", rettype = "fasta", retmode = "text")
pig_record = SeqIO.read(pig, "fasta")

In [43]:
# 소(Cow)
cow = Entrez.efetch(db = "protein", id = "P01317.2", rettype = "fasta", retmode = "text")
cow_record = SeqIO.read(cow, "fasta")

In [44]:
# 닭(Chicken)
chicken = Entrez.efetch(db = "protein", id = "P67970.1", rettype = "fasta", retmode = "text")
chicken_record = SeqIO.read(chicken, "fasta")

```python

In [45]:
# 아미노산 기록을 서열 변수로 저장
human   = human_record.seq
pig     = pig_record.seq
cow     = cow_record.seq
chicken = chicken_record.seq

# 4종을 한 번에 모두 출력
print("🧍‍♂️ Human   :", human)
print("🐖 Pig     :", pig)
print("🐄 Cow     :", cow)
print("🐔 Chicken :", chicken)


🧍‍♂️ Human   : MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
🐖 Pig     : MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN
🐄 Cow     : MALWTRLRPLLALLALWPPPPARAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEGPQVGALELAGGPGAGGLEGPPQKRGIVEQCCASVCSLYQLENYCN
🐔 Chicken : MALWIRSLPLLALLVFSGPGTSYAAANQHLCGSHLVEALYLVCGERGFFYSPKARRDVEQPLVSSPLRGEAGVLPFQQEEYEKVKRGIVEQCCHNTCSLYQLENYCN


In [47]:
# 각 종의 인슐린 아미노산 개수 확인
print("Human   :", len(human))
print("Pig     :", len(pig))
print("Cow     :", len(cow))
print("Chicken :", len(chicken))


Human   : 110
Pig     : 108
Cow     : 105
Chicken : 107


```python

In [71]:
# 서열 정렬 (Needleman–Wunsch 알고리즘)
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

# globalxx (match=1, mismatch=0, gap=0)
alignments_human_pig = pairwise2.align.globalxx(human, pig)
for align in alignments_human_pig[0:3]:
  print(format_alignment(*align))


MALWM-RLLPLLALLALWGPD-PA-A-AFVNQHLCGSHLVEALYLVCGERGFFYTPKT-RREAEDL--QV-GQ-VELGGGP-GAGSLQP-LALEGSL--QKRGIVEQCCTSICSLYQLENYCN
||||  ||||||||||||    || | ||||||||||||||||||||||||||||||  |||||    |  |  ||||||  | | ||  |||||    ||||||||||||||||||||||||
MALW-TRLLPLLALLALW---APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK-ARREAE--NPQ-AG-AVELGGG-LG-G-LQ-ALALEG--PPQKRGIVEQCCTSICSLYQLENYCN
  Score=95

MALWMRLLPLLALLALWGPD-PA-A-AFVNQHLCGSHLVEALYLVCGERGFFYTPKT-RREAEDL--QV-GQ-VELGGGP-GAGSLQP-LALEGSL--QKRGIVEQCCTSICSLYQLENYCN
||||.||||||||||||    || | ||||||||||||||||||||||||||||||  |||||    |  |  ||||||  | | ||  |||||    ||||||||||||||||||||||||
MALWTRLLPLLALLALW---APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK-ARREAE--NPQ-AG-AVELGGG-LG-G-LQ-ALALEG--PPQKRGIVEQCCTSICSLYQLENYCN
  Score=95

MALWM-RLLPLLALLALWGPDPA-A-AFVNQHLCGSHLVEALYLVCGERGFFYTPKT-RREAEDL--QV-GQ-VELGGGP-GAGSLQP-LALEGSL--QKRGIVEQCCTSICSLYQLENYCN
||||  ||||||||||||  .|| | ||||||||||||||||||||||||||||||  |||||    |  |  ||||||  | | ||  |||||    ||||||||||||||

```python

In [70]:
# globalms(match, mismatch, gap_open, gap_extend)
match_score         =  2
mismatch_penalty    = -1
gap_open_penalty    = -0.5
gap_extend_penalty  = -0.1

alignments_human_pig_ms = pairwise2.align.globalms(
    human, pig, match_score,
    mismatch_penalty,
    gap_open_penalty,
    gap_extend_penalty
)

print(format_alignment(*alignments_human_pig_ms[0]))


MALWM-RLLPLLALLALWGPD-PA-A-AFVNQHLCGSHLVEALYLVCGERGFFYTPKT-RREAEDL--QV-GQ-VELGGGP-GAGSLQP-LALEGSL--QKRGIVEQCCTSICSLYQLENYCN
||||  ||||||||||||    || | ||||||||||||||||||||||||||||||  |||||    |  |  ||||||  | | ||  |||||    ||||||||||||||||||||||||
MALW-TRLLPLLALLALW---APAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPK-ARREAE--NPQ-AG-AVELGGG-LG-G-LQ-ALALEG--PPQKRGIVEQCCTSICSLYQLENYCN
  Score=178.4



```python

============================================================

## CHAPTER 7 정리: 서열 정렬 (Sequence Alignment)

============================================================

✅ **배운 내용**

- NCBI Entrez를 이용해 사람·돼지·소·닭의 인슐린 단백질 서열을 불러올 수 있다.  
- Biopython의 `pairwise2` 모듈을 이용해 **단백질 서열 간 전역 정렬(Global alignment)** 을 수행할 수 있다.  
- `globalxx()` 함수는 **Needleman–Wunsch 알고리즘**을 사용하여 서열 전체를 정렬한다.  
- `globalms()` 함수를 사용하면 **match, mismatch, gap 점수**를 직접 설정하여 정렬 품질을 조정할 수 있다.  
- `format_alignment()` 함수를 통해 정렬 결과를 보기 좋게 출력하고,  
  **일치(`|`) / 불일치 / 갭(`-`)** 위치를 한눈에 확인할 수 있다.  

---

🧬 **생명과학 응용**

- **인슐린 단백질 비교** — 사람과 돼지 인슐린 서열이 거의 동일함을 확인  
- **변이 위치 파악** — 종 간 아미노산 치환 위치를 통해 기능 보존 부위를 추정  
- **단백질 진화 연구** — 서열 정렬 결과를 이용해 진화적 유사성을 정량적으로 비교  
